# 5 오차역전파법  
  
## 5.1 계산 그래프

계산 그래프: 계산 과정을 그래프로 나타낸 것  

그래프는 노드(node), 엣지(edge)로 표현

### 5.1 계산 그래프로 풀다  
계산 그래프는 계산 과정을 노드와 화살표로 표현  
노드는 원으로 표기, 안에는 연산 내용을 적는다.  

1. 계산 그래프 구성  
2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행  -> 순전파(forward propagation)  

순전파가 존재하므로 역전파도 존재한다. 역전파는 미분을 계산할 때 중요한 역할을 한다.  

### 5.1.2 국소적 계산
계산 그래프를 보면 자신과 직접적으로 관계된 정보만으로 결과를 출력할 수 있다.  
ex) 그림에서 사과 2개 사면 사과 x 2만 하면 된다. 다른건 안 한다.  


### 5.1.3 왜 계산 그래프로 푸는가?  
계산 그래프의 이점  
1) 국소적 계산 -> 전체가 아무리 복잡해도 각 노드에서는 단순한 계산에 집중하여 문제를 단순화 할 수 있다.
2) 중간 계산 결과를 모두 보관 가능  <b>
3) 역전파를 통해 미분을 효율적으로 계산할 수 있다.  (그림 5-5 참고)

## 5.2 연쇄법칙  
'국소적 미분'을 전달하는 원리는 연쇄법칙(chain rule)에 따른 것  

#### 5.2.1 계산 그래프의 역전파  
국소적인 미분을 상류에서 전달된 값(그림 5-6에서 E)에 곱해 앞쪽 노드로 전달한다.  

### 5.2.2 연쇄 법칙이란?  
합성 함수: 여러 함수로 구성된 함수  
연쇄법칙은 합성 함수의 미분에 대한 성질이다. 다음과 같이 정의할 수 있다.  
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;_합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다._  
<br>  

## 5.3 역전파  

### 5.3.1 덧셈 노드의 역전파  
덧셈 노드 역전파는 입력 신호를 다음 노드로 출력할 뿐이므로 그대로 다음 노드로 전달한다.  

### 5.3.2 곱셈 노드의 역전파  
곱셈 노드 역전파는 상류의 값에 순전파 때의 입력 신호들을 '서로 바꾼 값'을 곱해서 하류로 보낸다.  
덧셈의 역전파에서는 상류의 값을 그대로 흘려보내서 순방향 입력 신호의 값은 필요하지 않다.  
반면에 곱셈의 역전파는 순방향 입력 신호의 값이 필요하다. 곱셈 노드를 구현할 때 순전파의 입력신호를 변수에 저장해둔다.  



In [1]:
# 5.4 단순한 계층 구현 -> 계층: 신경망의 기능 단위

# 5.4.1 곱셈 계층
class MuLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y): # 순전파
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout): # 역전파              # dout(미분)
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x # y와 x를 바꾼다.
        
        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MuLayer()
mul_tax_layer = MuLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [6]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [7]:
# 5.4.2 덧셈 계층
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [8]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MuLayer()
mul_orange_layer = MuLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MuLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num) # (1)
orange_price = mul_orange_layer.forward(orange, orange_num) # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price) # (3)
price = mul_tax_layer.forward(all_price, tax) # (4)

# 역전파
dpirce = 1
dall_price, dtax = mul_tax_layer.backward(dprice) # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) #(1)

print(price) # 715
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


In [9]:
# 5.5 활성화 함수 계층 구현하기

# 5.5.1 ReLU 계층
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

mask는 True/False로 구성된 넘파이 배열

In [10]:
# 5.5.2 Sigmoid 계층
import numpy as np

class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

## 5.6 Affine / Softmax 계층 구현하기  

### 5.6.1 Affine 계층  
신경망의 순전파 때 수행하는 행렬의 내적은 기하학에서는 어파인 변환(Affine transformation)이라 한다.  
책에서는 어파인 변환을 수행하는 처리를 'Affine 계층'이라는 이름으로 구현한다.  

### 5.6.2 배치용 Affine 계층  
데이터 N개를 묶어 순전파하는 경우, 즉 배치용 Affine 계층  


In [12]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

In [13]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [15]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [16]:
dY = np.array([[1, 2, 3,], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [22]:
dB = np.sum(dY, axis = 0) # 0이면 열방향으로 합, 1이면 행방향으로 합
dB

array([5, 7, 9])

In [5]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = B
        self.x = None
        self.dw = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.w) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx

### 5.6.3 Softmax-with-Loss 계층  
신경망에서 수행하는 작업은 학습과 추론 두 가지다.  
추론할 때는 일반적으로 Softmax 계층을 사용하지 않는다.  
신경망은 추론할 때는 마지막 Affine 계층의 출력을 인식 결과로 이용한다.  
신경망에서 정규화하지 않는 출력 결과(그림 5-28에서는 Softmax 앞의 Affine 계층의 출력)를 '점수'라 한다.  
즉, 신경망 추론에서 답을 하나만 내는 경우에는 가장 높은 점수만 알면 되므로 Softmax 계층은 필요 없다.  
신경망을 학습할 때는 Softmax 계층이 필요하다.  

소프트맥스 함수의 손실 함수로 교차 엔트로피 오차를 사용하면 역전파가 말끔히 떨어진다.  

회귀의 출력층에서 사용하는 항등함수의 손실함수로 평균 제곱 오차를 이용하면 역전파의 결과가 말끔히 떨어진다.

In [6]:
from functions import *

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실
        self.y = None    # softmax의 출력
        self.x = None    # 정답 레이블(원-핫 벡터)
        
    def forward(self, x, t):
        self.t = t
        self. y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx=(self.y - self.t) / batch_size
        
        return dx

### 5.7.1 신경망 학습의 전체 그림  

신경망 학습의 순서  

* 전제  
신경망에서는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 한다.  

* 1단계-미니배치  
훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니배치라 한다. 그 미니배치의 손실 함수를 줄이는 것이 목표  

* 2단계-기울기 산출  
미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시  

* 3단계-매개변수 갱신  
가중치 매개변수를 기울기 방향으로 아주 조금 갱신  

* 4단계-반복  
1~3단계 반복  

<br>
전 장에서는 수치 미분을 사용해서 기울기를 구했다. 수치 미분은 구현하기 쉽지만, 계산이 오래 걸렸다.  
오차역전파법을 이용하면 수치 미분보다 빠르게 기울기를 구할 수 있다.  

In [6]:
import numpy as np
from layers import *
from gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
    
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    def loss(self, x, t):       # x: 입력 데이터, t: 정답 레이블
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

오차역전법이 있으면 수치 미분이 쓸모 없는 것이 아니다.  
오차역전법을 정확히 구현했는지 확인하기 위해서는 수치 미분이 필요하다.  
수치미분과 오차역전법으로 구한 기울기가 일치함을 확인하는 작업을 기울기 확인(gradient check)이라고 한다.  

In [10]:
# 기울기 확인
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size = 784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다.
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:7.581209748527943e-07
b1:7.6027188125434954e-06
W2:6.070309449540768e-09
b2:1.3969978891054113e-07


In [16]:
### 5.7.4 오차역전파법을 사용한 학습 구현하기
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 1000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차역전파법으로 기울기를 구한다.
    grad = network.gradient(x_batch, t_batch)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.17656666666666668 0.1819
0.90285 0.907
